# 🎤 Whisper - Převod řeči na text (Speech-to-Text)

**Autor:** Praut s.r.o. - AI Integration & Business Automation

## Co se naučíte:
- Transkripce audio souborů
- Vícejazyčná transkripce
- Automatické titulky
- Zpracování schůzek a hovorů

In [ ]:
!pip install -q transformers accelerate torch librosa soundfile datasets

In [ ]:
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration
import torch
import librosa

device = 0 if torch.cuda.is_available() else -1
print(f"🖥️ Device: {'GPU' if device == 0 else 'CPU'}")

## 1. Základní transkripce s Whisper

In [ ]:
# Whisper pipeline
whisper = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",
    device=device
)

# Pro demo použijeme sample z datasetu
from datasets import load_dataset

# Načtení sample audio
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation[:1]")
sample = dataset[0]

print(f"📊 Audio info:")
print(f"   Sampling rate: {sample['audio']['sampling_rate']} Hz")
print(f"   Délka: {len(sample['audio']['array']) / sample['audio']['sampling_rate']:.1f} s")

# Transkripce
result = whisper(sample['audio']['array'])

print(f"\n🎤 Transkripce:")
print(f"   {result['text']}")

## 2. Různé velikosti Whisper modelů

In [ ]:
# Dostupné modely:
# - openai/whisper-tiny (39M parametrů)
# - openai/whisper-base (74M)
# - openai/whisper-small (244M)
# - openai/whisper-medium (769M)
# - openai/whisper-large-v3 (1.5B)

print("📊 Porovnání Whisper modelů:\n")
print("| Model | Parametry | VRAM | Rychlost | Přesnost |")
print("|-------|-----------|------|----------|----------|")
print("| tiny  | 39M       | ~1GB | Nejrychlejší | Základní |")
print("| base  | 74M       | ~1GB | Rychlý   | Dobrá    |")
print("| small | 244M      | ~2GB | Střední  | Velmi dobrá |")
print("| medium| 769M      | ~5GB | Pomalý   | Vynikající |")
print("| large | 1.5B      | ~10GB| Nejpomalejší | Nejlepší |")

In [ ]:
# Načtení většího modelu pro lepší přesnost
whisper_small = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    device=device
)

result_small = whisper_small(sample['audio']['array'])

print("🔍 Porovnání výsledků:")
print(f"   Base:  {result['text']}")
print(f"   Small: {result_small['text']}")

## 3. Transkripce s časovými značkami

In [ ]:
# Transkripce s timestamps
result_timestamps = whisper_small(
    sample['audio']['array'],
    return_timestamps=True
)

print("⏱️ Transkripce s časovými značkami:\n")

if 'chunks' in result_timestamps:
    for chunk in result_timestamps['chunks']:
        start = chunk['timestamp'][0] if chunk['timestamp'][0] else 0
        end = chunk['timestamp'][1] if chunk['timestamp'][1] else '?'
        print(f"   [{start:.1f}s - {end}s] {chunk['text']}")
else:
    print(f"   {result_timestamps['text']}")

## 4. Vícejazyčná transkripce

In [ ]:
# Whisper automaticky detekuje jazyk
# Můžeme ale specifikovat jazyk pro lepší výsledky

def transkribuj(audio_array, jazyk=None):
    """Transkribuje audio s volitelným určením jazyka."""
    
    generate_kwargs = {}
    if jazyk:
        generate_kwargs["language"] = jazyk
    
    result = whisper_small(
        audio_array,
        generate_kwargs=generate_kwargs
    )
    
    return result['text']

# Podporované jazyky (výběr)
print("🌍 Podporované jazyky Whisper:")
jazyky = [
    "english", "czech", "german", "french", "spanish",
    "italian", "polish", "portuguese", "russian", "chinese",
    "japanese", "korean", "arabic", "hindi", "turkish"
]
print(f"   {', '.join(jazyky)}")
print("   ... a dalších 80+ jazyků")

## 5. Generování titulků (SRT formát)

In [ ]:
def generuj_srt(audio_array, sampling_rate=16000):
    """Generuje titulky ve formátu SRT."""
    
    result = whisper_small(
        audio_array,
        return_timestamps=True,
        chunk_length_s=30
    )
    
    srt_lines = []
    
    if 'chunks' in result:
        for i, chunk in enumerate(result['chunks'], 1):
            start = chunk['timestamp'][0] if chunk['timestamp'][0] else 0
            end = chunk['timestamp'][1] if chunk['timestamp'][1] else start + 5
            
            # Formátování času pro SRT
            start_srt = f"{int(start//3600):02d}:{int((start%3600)//60):02d}:{int(start%60):02d},{int((start%1)*1000):03d}"
            end_srt = f"{int(end//3600):02d}:{int((end%3600)//60):02d}:{int(end%60):02d},{int((end%1)*1000):03d}"
            
            srt_lines.append(f"{i}")
            srt_lines.append(f"{start_srt} --> {end_srt}")
            srt_lines.append(chunk['text'].strip())
            srt_lines.append("")
    
    return "\n".join(srt_lines)

# Generování SRT
srt = generuj_srt(sample['audio']['array'])
print("📝 Vygenerované SRT titulky:\n")
print(srt)

## 6. Zpracování audio souborů

In [ ]:
import soundfile as sf
import numpy as np

def transkribuj_soubor(cesta_k_souboru):
    """Transkribuje audio soubor (MP3, WAV, FLAC...)."""
    
    # Načtení audio
    audio, sr = librosa.load(cesta_k_souboru, sr=16000)
    
    # Transkripce
    result = whisper_small(
        audio,
        return_timestamps=True
    )
    
    return {
        'text': result['text'],
        'chunks': result.get('chunks', []),
        'duration': len(audio) / sr
    }

# Demo - vytvoříme testovací audio
print("📁 Funkce transkribuj_soubor() připravena.")
print("   Použití: transkribuj_soubor('cesta/k/audio.mp3')")
print("   Podporované formáty: MP3, WAV, FLAC, OGG, M4A")

## 7. Dávkové zpracování více souborů

In [ ]:
import pandas as pd
from pathlib import Path

def zpracuj_slozku(cesta_slozky, pripony=['.mp3', '.wav', '.flac']):
    """Zpracuje všechny audio soubory ve složce."""
    
    vysledky = []
    slozka = Path(cesta_slozky)
    
    soubory = [f for f in slozka.iterdir() if f.suffix.lower() in pripony]
    
    for soubor in soubory:
        print(f"   Zpracovávám: {soubor.name}")
        try:
            result = transkribuj_soubor(str(soubor))
            vysledky.append({
                'soubor': soubor.name,
                'delka_s': result['duration'],
                'text': result['text'],
                'status': 'OK'
            })
        except Exception as e:
            vysledky.append({
                'soubor': soubor.name,
                'delka_s': 0,
                'text': '',
                'status': f'Error: {str(e)}'
            })
    
    return pd.DataFrame(vysledky)

print("📁 Funkce zpracuj_slozku() připravena.")
print("   Použití: zpracuj_slozku('/cesta/ke/slozce')")

## 8. Překlad a transkripce zároveň

In [ ]:
# Whisper umí přímo překládat do angličtiny
whisper_translate = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    device=device
)

def transkribuj_a_preloz(audio_array):
    """Transkribuje a přeloží do angličtiny."""
    
    # Původní transkripce
    original = whisper_translate(audio_array)
    
    # Překlad do angličtiny
    translated = whisper_translate(
        audio_array,
        generate_kwargs={"task": "translate"}
    )
    
    return {
        'original': original['text'],
        'english': translated['text']
    }

print("🌍 Funkce transkribuj_a_preloz() připravena.")
print("   Přeloží audio z libovolného jazyka do angličtiny.")

## 9. Zpracování schůzek

In [ ]:
def zpracuj_schuzku(audio_array):
    """Zpracuje nahrávku schůzky a vytvoří strukturovaný výstup."""
    
    # Transkripce s timestamps
    result = whisper_small(
        audio_array,
        return_timestamps=True,
        chunk_length_s=30
    )
    
    # Struktura výstupu
    output = {
        'full_transcript': result['text'],
        'duration_minutes': len(audio_array) / 16000 / 60,
        'segments': []
    }
    
    if 'chunks' in result:
        for chunk in result['chunks']:
            output['segments'].append({
                'start': chunk['timestamp'][0],
                'end': chunk['timestamp'][1],
                'text': chunk['text'].strip()
            })
    
    return output

# Demo
meeting = zpracuj_schuzku(sample['audio']['array'])

print("📋 Zápis ze schůzky:\n")
print(f"Délka: {meeting['duration_minutes']:.1f} minut")
print(f"\nTranskript:\n{meeting['full_transcript']}")

## 10. Praktická automatizace: Voice Notes

In [ ]:
from datetime import datetime

class VoiceNotesProcessor:
    def __init__(self, model_name="openai/whisper-small"):
        self.whisper = pipeline(
            "automatic-speech-recognition",
            model=model_name,
            device=device
        )
        self.notes = []
    
    def add_note(self, audio_array, title=None):
        """Přidá hlasovou poznámku."""
        result = self.whisper(audio_array, return_timestamps=True)
        
        note = {
            'id': len(self.notes) + 1,
            'title': title or f"Note {len(self.notes) + 1}",
            'timestamp': datetime.now().isoformat(),
            'text': result['text'],
            'duration': len(audio_array) / 16000
        }
        
        self.notes.append(note)
        return note
    
    def search_notes(self, keyword):
        """Vyhledá v poznámkách."""
        return [n for n in self.notes if keyword.lower() in n['text'].lower()]
    
    def export_markdown(self):
        """Exportuje poznámky do Markdown."""
        md = "# Voice Notes\n\n"
        for note in self.notes:
            md += f"## {note['title']}\n"
            md += f"*{note['timestamp']}* ({note['duration']:.1f}s)\n\n"
            md += f"{note['text']}\n\n---\n\n"
        return md

# Demo
processor = VoiceNotesProcessor()
processor.add_note(sample['audio']['array'], "Meeting notes")

print("📝 Voice Notes System:")
print(processor.export_markdown())

---
## 🏁 Shrnutí

- ✅ Whisper pro přesnou transkripci
- ✅ Podpora 90+ jazyků
- ✅ Automatické titulky (SRT)
- ✅ Dávkové zpracování souborů
- ✅ Překlad z řeči do angličtiny

**Další notebook:** Computer Vision - klasifikace a detekce